In [1]:
# Dependencies and Setup
import pandas as pd

In [2]:
# File to Load 
hero_path = "HeroesOfPymoli/Resources/purchase_data.csv"


In [3]:
# Read Purchasing File and store into Pandas data frame
hero_df=pd.read_csv(hero_path)

## Just looking at what I am working with.

In [4]:
hero_df

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44
...,...,...,...,...,...,...,...
775,775,Aethedru70,21,Female,60,Wolf,3.54
776,776,Iral74,21,Male,164,Exiled Doomblade,1.63
777,777,Yathecal72,20,Male,67,"Celeste, Incarnation of the Corrupted",3.46
778,778,Sisur91,7,Male,92,Final Critic,4.19


In [5]:
hero_df.dtypes

Purchase ID      int64
SN              object
Age              int64
Gender          object
Item ID          int64
Item Name       object
Price          float64
dtype: object

In [6]:
hero_df.shape

(780, 7)

In [7]:
hero_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 780 entries, 0 to 779
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Purchase ID  780 non-null    int64  
 1   SN           780 non-null    object 
 2   Age          780 non-null    int64  
 3   Gender       780 non-null    object 
 4   Item ID      780 non-null    int64  
 5   Item Name    780 non-null    object 
 6   Price        780 non-null    float64
dtypes: float64(1), int64(3), object(3)
memory usage: 42.8+ KB


## Player Count

* Display the total number of players

In [8]:
#Create a dictionary (key is Total Players: value is the count of the total number 
# of unique players), convert it to a DataFrame, and print the new DF. 
tp_df=pd.DataFrame({"Total Players":[len(hero_df["SN"].value_counts())]})
tp_df

,Total Players
0,576


## Purchasing Analysis (Total) 
* Number of Unique Items
* Average Purchase Price
* Total Number of Purchases
* Total Revenue


In [9]:
# Create a new DataFrame with columns: Number of Unique Items, Average Price, 
# Number of Purchases, and Total Revenue. Print the DataFrame.  
pa_df=pd.DataFrame({"Number of Unique Items":[len(hero_df["Item Name"].value_counts())],
                    "Average Price":pd.Series(hero_df["Price"].mean()).map("${:,.2f}".format),
                    "Total Revenue":pd.Series(hero_df["Price"].sum()).map("${:,.2f}".format)
                   })
pa_df

,Number of Unique Items,Average Price,Total Revenue
0,179,$3.05,"$2,379.77"


## Gender Demographics:
* Percentage and Count of Male Players
* Percentage and Count of Female Players
* Percentage and Count of Other / Non-Disclosed


In [10]:
# Create new DataFrame of the count and percentage of each gender. Print DataFrame.
gender_df=pd.DataFrame(hero_df.drop_duplicates("SN")["Gender"].value_counts())
gender_df["Percentage of Players"]=pd.Series(round(gender_df['Gender']/len(hero_df['SN'].value_counts())*100,2)).map("{:,.2f}%".format)
gender_df=gender_df.rename({"Gender":"Total Count"},axis=1)
gender_df

,Total Count,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


## Purchasing Analysis (Gender)
* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender
* Create a summary data frame to hold the results
* Optional: give the displayed data cleaner formatting
* Display the summary data frame


In [11]:
# Calculate purchase count, avg. purchase price, total purchase per gender, 
# and average total purchase per person per gender.
gender_count = hero_df.groupby(['Gender']).count()['Price']
mean_pp = hero_df.groupby(['Gender']).mean()['Price']
gender_pt = hero_df.groupby(['Gender']).sum()['Price']
# Use the gender count from the previous DF to calculate Avg Total Purchase per Person by Gender. 
mean_ip = gender_pt/gender_df['Total Count']

# Create my DF for Purchase Analysis by Gender                                               
gender_pa_df = pd.DataFrame({"Purchase Count":gender_count,
                             "Average Purchase Price":mean_pp,
                             "Total Purchase Value":gender_pt,
                             "Avg Total Purchase per Person":mean_ip})

# Format dollars and decimals
gender_pa_df['Average Purchase Price']=gender_pa_df['Average Purchase Price'].map("${:,.2f}".format)
gender_pa_df['Total Purchase Value']=gender_pa_df['Total Purchase Value'].map("${:,.2f}".format)
gender_pa_df['Avg Total Purchase per Person']=gender_pa_df['Avg Total Purchase per Person'].map("${:,.2f}".format)

gender_pa_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics:
* Establish bins for ages
* Categorize the existing players using the age bins. Hint: use pd.cut()
* Calculate the numbers and percentages by age group
* Create a summary data frame to hold the results
* Optional: round the percentage column to two decimal points
* Display Age Demographics Table

In [12]:
# Ran the min and max functions to verify how many bins I should have. 
# Set up my bins and their labels
bins=[0, 9.99, 14.99, 19.99, 24.99, 29.99, 34.99, 39.99, 100]
labels=["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39","40+"]

#Place the data in a new column inside the existing DF
hero_df["Age Ranges"]=pd.cut(hero_df["Age"], bins, labels=labels)

# Calculations for the new DF we want to build
age_totals= hero_df.drop_duplicates("SN")["Age Ranges"].value_counts()
age_percent= (round(age_totals/len(hero_df['SN'].value_counts())*100,2)).map("{:,.2f}%".format)

# New DF
age_df = pd.DataFrame({"Total Count" : age_totals,
                       "Percentage of Players" :age_percent
                      },index=["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39","40+"])
age_df

,Total Count,Percentage of Players
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)
* Bin the purchase_data data frame by age
* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below
* Create a summary data frame to hold the results
* Optional: give the displayed data cleaner formatting
* Display the summary data frame

In [13]:
# Since we have the bins set up and sorted above I don't need to do that again here. 

# Calculate purchase count, avg. purchase price, total purchase per age group, 
# and average total purchase per person per age group. Same as in the gender analysis.
age_count = hero_df.groupby(['Age Ranges']).count()['Price']
mean_pp = hero_df.groupby(['Age Ranges']).mean()['Price']
age_pt = hero_df.groupby(['Age Ranges']).sum()['Price']
mean_ip = age_pt/(age_df['Total Count'])

# Create my DF for Purchase Analysis by Age                                               
age_pa_df = pd.DataFrame({"Purchase Count":age_count,
                             "Average Purchase Price":mean_pp,
                             "Total Purchase Value":age_pt,
                             "Avg Total Purchase per Person":mean_ip})

# Format dollars and decimals
age_pa_df['Average Purchase Price']=age_pa_df['Average Purchase Price'].map("${:,.2f}".format)
age_pa_df['Total Purchase Value']=age_pa_df['Total Purchase Value'].map("${:,.2f}".format)
age_pa_df['Avg Total Purchase per Person']=age_pa_df['Avg Total Purchase per Person'].map("${:,.2f}".format)

age_pa_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Ranges,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders:
* Identify the the top 5 spenders in the game by total purchase value, then list (in a table):
* SN
* Purchase Count
* Average Purchase Price
* Total Purchase Value


In [14]:
# Set up my values.
spend_count=hero_df.groupby(["SN"]).count()["Price"]
mean_pp=hero_df.groupby(["SN"]).mean()["Price"]
spend_pt=hero_df.groupby(["SN"]).sum()["Price"]

# Set up my DataFrame
spend_df= pd.DataFrame({
    "Purchase Count":spend_count,
    "Average Purchase Price":mean_pp,
    "Total Purchase Value":spend_pt  
})

# Sorted my DF in descending order.
spend_df=spend_df.sort_values("Total Purchase Value", ascending=False)

# Formatted my DF. 
spend_df['Average Purchase Price']=spend_df['Average Purchase Price'].map("${:,.2f}".format)
spend_df['Total Purchase Value']=spend_df['Total Purchase Value'].map("${:,.2f}".format)

# Print the top 5 spenders.
spend_df.head(5)


,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [15]:
# Taking a look at what the top spenders bought, their gender, and the purchase prices of their individual items. 
top_spenders_df = hero_df[hero_df["SN"]==list(spend_df.head(5).index)[0]]
for top_index in range(1,5):
    single_spender_df=hero_df[hero_df["SN"]==list(spend_df.head(5).index)[top_index]]
    top_spenders_df=pd.concat([single_spender_df,top_spenders_df])
top_spenders_df

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price,Age Ranges
148,148,Iskadarya95,20,Male,148,"Warmonger, Gift of Suffering's End",4.03,20-24
247,247,Iskadarya95,20,Male,82,Nirvana,4.90,20-24
318,318,Iskadarya95,20,Male,91,Celeste,4.17,20-24
128,128,Iral74,21,Male,58,"Freak's Bite, Favor of Holy Might",4.14,20-24
623,623,Iral74,21,Male,114,Yearning Mageblade,3.82,20-24
758,758,Iral74,21,Male,182,Toothpick,4.03,20-24
776,776,Iral74,21,Male,164,Exiled Doomblade,1.63,20-24
222,222,Chamjask73,22,Female,178,"Oathbreaker, Last Hope of the Breaking Storm",4.23,20-24
520,520,Chamjask73,22,Female,109,"Downfall, Scalpel Of The Emperor",4.76,20-24
564,564,Chamjask73,22,Female,52,Hatred,4.84,20-24


## Most Popular Items:
* Identify the 5 most popular items by purchase count, then list (in a table):
* Item ID
* Item Name
* Purchase Count
* Item Price
* Total Purchase Value


In [16]:
# Set up my values.
count=hero_df.groupby(["Item ID", "Item Name"]).count()["Purchase ID"]
price=hero_df.groupby(["Item ID","Item Name"]).mean()["Price"]
tpv=hero_df.groupby(["Item ID", "Item Name"]).sum()["Price"]

# Set up my DataFrame. 
count_df=pd.DataFrame({
    "Purchase Count":count,
    "Item Price":price,
    "Total Purchase Value":tpv
})

# Sort my DataFrame.
count_df=count_df.sort_values("Purchase Count", ascending=False)

# Apply formatting.
count_df['Item Price']=count_df['Item Price'].map("${:,.2f}".format)
count_df['Total Purchase Value']=count_df['Total Purchase Value'].map("${:,.2f}".format)

# Print the top 5.
count_df.head(5)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


## Most Profitable Items:
* Identify the 5 most profitable items by total purchase value, then list (in a table):
* Item ID
* Item Name
* Purchase Count
* Item Price
* Total Purchase Value


In [17]:
# Set up my values.
count=hero_df.groupby(["Item ID", "Item Name"]).count()["Purchase ID"]
price=hero_df.groupby(["Item ID", "Item Name"]).mean()["Price"]
tpv=hero_df.groupby(["Item ID", "Item Name"]).sum()["Price"]

# Set up the DataFrame.
count_df=pd.DataFrame({
    "Purchase Count":count,
    "Item Price":price,
    "Total Purchase Value":tpv
})

# Sort the DataFrame.
count_df=count_df.sort_values("Total Purchase Value", ascending=False)

# Format the DataFrame. 
count_df['Item Price']=count_df['Item Price'].map("${:,.2f}".format)
count_df['Total Purchase Value']=count_df['Total Purchase Value'].map("${:,.2f}".format)

# Print the top five. 
count_df.head(5)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
